In [1]:
import os
import pandas as pd 
from glob import glob

import sys
import time, re, pickle, itertools
import urllib3, json
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import re

import nltk
from __future__ import print_function, unicode_literals
from gensim.models.word2vec import Word2Vec
from sklearn.preprocessing import normalize as sknorm
import numpy as np
from operator import itemgetter
import html

from ckonlpy.tag import Twitter as ctwitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Twitter, Mecab,Kkma
from konlpy.utils import pprint

import chat_bot as cb
import Database_Handler as dh

ct = ctwitter()
ot = Twitter()
mecab = Mecab()
xxxx = ct._dictionary._pos2words
print (list(map(lambda x: len(xxxx[x]), xxxx)))

score_weights = {
    'num_nouns': -0.1,
    'num_words': -0.2,
    'no_noun': -1
}

def my_score(candidate):
    num_nouns = len([w for w,t in candidate if t == 'Noun'])
    num_words = len(candidate)
    no_noun = 1 if num_nouns == 0 else 0
    
    score = (num_nouns * score_weights['num_nouns'] 
             + num_words * score_weights['num_words']
             + no_noun * score_weights['no_noun'])
    return score

ct.set_selector(score_weights, my_score)

[512, 1464079, 5532]
[('이', 'Noun'), ('것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]
best: [('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]


In [2]:
ext_Keyword_Daum_mecab = glob('./data/ko/mecab-daum*')
ext_Keyword_Daum_ckonlpy = glob('./data/ko/ckonlpy-daum*')

In [3]:
ckonlpyDict = dict()
for file in ext_Keyword_Daum_ckonlpy:
    ckonlpyDict.update(pickle.load(open(file,'rb')))
mecabDict = dict()
for file in ext_Keyword_Daum_mecab:
    mecabDict.update(pickle.load(open(file,'rb')))

In [4]:
#site = 'Naver'
site = 'daum'
if site == 'daum':
    collection = 'newsDaum'
elif site.lower() == 'naver':
    collection = 'newsNaver'
mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
slack = cb.Slacker(cb.slacktoken())
useCollection = dh.Use_Collection(useDb, collection)

In [5]:
def Check(list1, list2):
    first_keywords = list(filter(lambda x: x[0] == x[1], list(itertools.product(list1, list2))))
    first_keywords = set(map(lambda x: x[0], first_keywords))
    second_keywords = list(filter(lambda x: x[0] != x[1], list(itertools.product(list1, list2))))
    second_keywords = list(filter(lambda x: abs(len(max(x, key = len)) - len(''.join(re.findall('['+x[0]+']', x[1])).strip())) == 1,
                                  second_keywords))
    second_keywords = set(map(lambda x: max(x, key = len), second_keywords))
    return first_keywords|second_keywords
    

In [6]:
dataList = useCollection.find({'site':site})
i = 0 
for data in dataList:
    print (data['title'])
    print (data['link'])
    idIs = data['_id']._ObjectId__id.hex()
    outcome_ckonlpy = list(map(lambda x: ' '.join(list(dict.fromkeys(x.split(' ')))).strip(), ckonlpyDict[idIs]))
    outcome_ckonlpy2 = list(dict.fromkeys(outcome_ckonlpy + list(map(lambda y: ''.join(y), list(map(lambda x: x.split(' '), outcome_ckonlpy))))))
    outcome_mecab = list(map(lambda x: ' '.join(list(dict.fromkeys(x.split(' ')))).strip(), mecabDict[idIs]))
    outcome_mecab2 = list(dict.fromkeys(outcome_mecab + list(map(lambda y: ''.join(y), list(map(lambda x: x.split(' '), outcome_mecab))))))
    keywords = Check(outcome_ckonlpy2, outcome_mecab2)
    k2 = list(filter(lambda idx: abs(len(''.join(re.findall('['+idx[0]+']',idx[1])).strip())-len(min(idx, key = len))) == 0, list(itertools.combinations(keywords, 2))))
    k2 = set(map(lambda x: min(x, key = len), k2))
    list(map(lambda x: keywords.remove(x), k2))
    print (data['keywords'])
    print (keywords)
    print ()

김국진·노홍철·한혜진 '한명회', 오늘(5일) 9회만에 종영
http://v.media.daum.net/v/20171205151743503
['노홍철', '한명회', '한혜진', '종영', '편성시간']
{'이름을', '한명회', '녹화', '출연자들', '특집'}

'몸 만드는 게 우선' 류중일 감독, 캠프 명단에 신인 없다
http://v.media.daum.net/v/20171209104014068
['류중일']
{'명단', '신인', '오키나와 캠프', '류중일 감독'}

[어제TV]첫방 '저글러스' 男보스 女비서, 시대착오적 설정 '불편'
http://v.media.daum.net/v/20171205061623329
['불륜', '비', '첫방', '최다니엘', '최대철']
{'아내', '봉상무', '무릎', '불륜', '보스', '비서', '설정'}

[Oh! 무비] '기억의 밤' 장항준 감독은 어떻게 골리앗을 무찔렀나
http://v.media.daum.net/v/20171205140114189
['기억의밤', '시나리오', '리얼리티', '강하늘', '김무열']
{'영화', '스릴러', '시나리오', '기억의', '드라마', '장항준 감독'}

[RE:TV]'섬총사' 강호동X정상훈, 80cm 대어 낚아 '낚시TV 따로없네'
http://v.media.daum.net/v/20171205065011570
['강호동', '정상훈', '조세호', '어청도', '달타냥']
{'낚시', '섬총사', '강호동과 정상훈', '어청도', '조세호'}

'범죄도시'는 왜 불법 다운과의 전쟁을 선포했나
http://v.media.daum.net/v/20171204154759778
['윤계상', '마동석', '강경대응', 'sns']
{'행위', '불법', '범죄 도시', '영화', '유포자들'}

'다둥이 아빠' 박지헌, 여섯째 출산 앞둔 아내에 "정말 고마워" 애틋
http://v.media.daum.net/v/20171211083610397
['출산', '혼인신

NaN
{'울스의', '농구', '미국', '워싱턴', '필리핀', '한국인'}

[단독] '청와대 불상'에 절하는 데라우치 총독 사진이 나왔다
http://v.media.daum.net/v/20171203175611950
['경주시', '관저', '청와대']
{'배례', '데라우치', '총독', '청와대 불상', '사진들', '개안식'}

'인천 낚싯배 전복 사고' 시화병원 의료진 브리핑 (전문)
http://v.media.daum.net/v/20171203132400466
['시화병원', '전복사고', '낚싯배']
{'상태', '사망', '인터뷰', '경찰', '검사', '환자분들', '상황', '구조'}

[영흥도 생존자 인터뷰] "튕겨져 나가 스티로폼 잡고 버텼다"
http://v.media.daum.net/v/20171203160120759
['이슈 · 영흥도 낚싯배 전복사고', '스티로폼', '영흥도', '해경']
{'선미', '인천', '스티로폼', '해경', '사고', '상황', '구조'}

SAT 만점 美 하버드대 유학생, 한국 돌아와 해병대 입대
http://v.media.daum.net/v/20171203091829855
['해병대', '하버드대', '유학생']
{'하버드대 컴퓨터공학과', '입대', '이병은', '미국', '해병대입대'}

"재난 컨트롤타워는 靑"..文대통령, 주말없이 '기민 대응'
http://v.media.daum.net/v/20171203150915767
['이슈 · 영흥도 낚싯배 전복사고', '컨트롤타워', '해경', '낚싯배']
{'대통령', '해경', '현장', '청와대', '구조', '보고'}

英 언론들 한 목소리, 손흥민 '스토크전 선발'
http://v.media.daum.net/v/20171209060655078
['손흥민']
{'손흥민의', '선발', '기록', '토트넘', '스토크', '경기', '영국'}

'런닝맨' 송지효, 몸무게 공개 위기에 당황 "옷 입어서 그래"
http://v.media.d

['이슈 · 국정원·군 정치개입 의혹', '우병우', '구속영장', '최윤수']
{'구속 영장', '법원', '차장', '기각', '수석', '수사', '검찰'}

[Oh!쎈 톡] '나혼자' PD "비, 이시언과 절친..10년 만에 노래방行"
http://v.media.daum.net/v/20171201152048292
['비', '이시언', '나혼자', '한혜진', '예고편']
{'예고편', '혼자', '공개', '이시언', '산다'}

[TV와치]'전지적 참견시점' 이재진, 독특함과 무례함은 다르다
http://v.media.daum.net/v/20171201133506384
['나 혼자 산다', '이재진', '이영자', '전현무', '젝스키스']
{'전지적', '이재진의', '방송', '이영자', '모습', '질문', '참견'}

[러시아월드컵 조추첨]개최국 러시아+이웃 일본, 최고의 행운 잡았다
http://v.media.daum.net/v/20171202011507212
NaN
{'일본', '월드컵', '편성', '조추첨', '러시아', '본선', 'FIFA 랭킹'}

트루아 회장 "석현준, 당연히 만족스럽다"
http://v.media.daum.net/v/20171212025211767
NaN
{'감독', '트루아', '석현준', '공격수', '출전', '활약', '가르시아', '프랑스', '영입'}

해경, 낚싯배 충돌 급유선 선장·선원 긴급체포(속보)
http://v.media.daum.net/v/20171203204211721
['이슈 · 영흥도 낚싯배 전복사고', '해경', '급유선', '낚싯배']
{'선장 선원', '충돌 급유선', '급유선 선장'}

[단독인터뷰] 김예령 "손녀 아닌 손자..사위 윤석민, 내년 MVP 되길"
http://v.media.daum.net/v/20171201101735646
['윤석민', '김수현', '결혼', '환희', '결혼식']
{'김예령', '축복', 'MVP', '활약', '한국', '윤석민', '손자'

['이슈 · 2018 예산안 본회의 통과', '한국당', '본회의', '의원총회']
{'의원 총회', '한국당 의원들', '본회의장'}

김문호-나경민-이병규 경쟁, '국대급' 롯데 외야는 전쟁터
http://v.media.daum.net/v/20171206072906455
['김문호', '이병규', '나경민']
{'민병헌', '김문호', '손아섭', '경쟁', '이대호', '이병규', '포지션', '외야', '롯데'}

밀란의 중국 자본, 알고 보니 막대한 '빚'
http://v.media.daum.net/v/20171201085119265
NaN
{'전문가들', '투자자', 'AC 밀란', '리용'}

[시나쿨파]끔찍한 중국 화장실, 그런 화장실도 없는 인도
http://v.media.daum.net/v/20171203083011370
['인도', '화장실', '칸막이']
{'인도', '여성', '용변', '중국 화장실', '해결', '좌변기'}

'평창 퇴출' 러시아, 미국과 날선 대립..FIFA는 '전전긍긍'
http://v.media.daum.net/v/20171206164721183
NaN
{'러시아의 도핑', 'IOC', '도핑 스캔들', 'WADA'}

前 MBC 아나운서 서현진, 5살 연상 의사와 9일 웨딩마치(전문)
http://v.media.daum.net/v/20171204095107254
['서현진', '결혼', 'ebs', '교양프로그램', '백년가약']
{'소속사', '결혼', '의사', '서현진씨', '실버 스톤', '연상'}

'셋째 출산' 유리 "저 건강해요..두 딸들도 보고싶다"
http://v.media.daum.net/v/20171205090723068
['보고싶다', '출산', '결혼', '산부인과', '혼성그룹']
{'공개', '모습', '유리', '사진'}

[인터뷰] 안현수 "'개인자격 출전'에 강경했던 러 정부, 지금은.."
http://v.media.daum.net/v/20171206222304145
['빅토르 

NaN
{'SK', 'LG', '김광현', '재활', '좌완', 'KBO 리그', '2018'}

[머니무브] 금리 인상 후폭풍..시중 자금 움직인다
http://v.media.daum.net/v/20171203071404910
['이슈 · 한은, 기준금리 인상', '금리인상', '예금금리', '대출금리']
{'자금', '시중', '예금', '대출금리', '인상', '금융', '상승'}

국민의당, 박주원 'DJ 비자금의혹 제보'에 발칵..호남계 맹비난
http://v.media.daum.net/v/20171208105019895
["이슈 · 'DJ 비자금 의혹' 제보 파문", '최고위원', '박주원', '국민의당']
{'최고 위원', '대표', '의혹', '제보', '의원', '검찰'}

영흥도 낚싯배 전복 8명 사망 2명 실종.."구명조끼 다 착용"(종합)
http://v.media.daum.net/v/20171203115948391
['이슈 · 영흥도 낚싯배 전복사고', '낚싯배', '영흥도', '급유선']
{'낚싯배', '경비정', '급유선', '현장', '사고', '신고', '오전', '구조'}

김광현이 돌아온다, 에이스 합류는 좋은데.. 'SK의 고민'
http://v.media.daum.net/v/20171206161200844
['김광현']
{'포스트시즌', '복귀', '에이스', '김광현', '100'}

中 매체 "산둥 루넝, 이동국 코치로 영입 의향"
http://v.media.daum.net/v/20171207123958487
NaN
{'구단', '중국', '산둥 루넝', '이동국', '영입'}

'냉부해' 고든램지vs이연복 맞대결 성사, 이연복 "은퇴시키겠다"
http://v.media.daum.net/v/20171211100415363
['이연복', '고든램지', '대결', '오승환', '15분']
{'이연복 셰프', '대결', '요리', '고든램지'}

효연, 초고층 호화 아파트에 외제차+캠핑 트레일러까지(살미도)
http://v.

...

['정채연', '장미희', '같이살래요', '유동근', '윤창범']
{'정채연', '미연', '배우 유인혁'}

[이슈&톡] '마약 스캔들' 이어 싸이와 결별설, YG '흔들'
http://v.media.daum.net/v/20171227123503942
['yg', '빅뱅', 'g-dragon', '태양', '빅뱅탑']
{'태양', '마약', 'YG', '싸이', '지드래곤', '제작', '방송', '빅뱅'}

'내남자' 두 송창의, 신분 바꾸기로 했다..박정아 알고 '충격'[종합]
http://v.media.daum.net/v/20171227202610985
['박정아', '송창의', '강재욱', '강인']
{'지섭', '강재욱', '며칠', '호주'}

'1987', 엔딩크레딧이 끝나도 한참을 일어서지 못한 이유
http://v.media.daum.net/v/20171227175502694
['엔딩크레딧', '한병용', '이한열열사', '박종철', '정덕현']
{'마음', '이야기', '연희', '민주화', '영화', '대학', '1987년', '당대'}

[夜TV] 입조심 예능 '강식당', 이수근 식당을 언급하다
http://v.media.daum.net/v/20171227065036304
['이수근', '신서유기3', '강호동', '송민호', '나영석pd']
{'송민호', '나영석', '영업', '이수근', '강호동', '강식당', '신서유기'}

[단독]조세호, 현 소속사와 재계약..남희석·남창희 함께
http://v.media.daum.net/v/20171227105556783
['남창희', '남희석', '공채', '개그맨', '무한도전']
{'남희석남창희', '재계약', '개그맨', '사람', '남희석 조세호'}

[SC초점] '감빵생활'vs'화유기' 똑같은 휴방, 엇갈린 반응 왜?
http://v.media.daum.net/v/20171227154910638
['화유기', '감빵생활', '슬기로운감빵생활', '신원호', '방송사고']
{'감빵 

['포체티노', '해리 케인']
{'해트트릭', '포체티노', '감독', '기록', '2017년', 'EPL', '케인', '리그'}

[골프스타 1막2장] 2. 송민지 "음식점 대박 터뜨려야죠"
http://v.media.daum.net/v/20171227093552463
NaN
{'마음', '송민지', '냉면', '투어', '30', '엄마'}

크리스마스에도 훈련한 크로캅..8연승 도전
http://v.media.daum.net/v/20171227055609764
NaN
{'토너먼트', 'UFC', '경기', '31일', '라이진', '크로캅'}

TEX 언론, "추신수 부활했지만 팀 리빌딩에 방해"
http://v.media.daum.net/v/20171227060418002
['추신수']
{'리빌딩', '추신수', '트레이드', '텍사스', '댈러스', '한국'}

잠시 자리비우는 박진태, 새 마당쇠 찾아야 할 KIA 불펜
http://v.media.daum.net/v/20171227153901291
['박진태']
{'공백', '역할', 'KIA', '박진태', '마당쇠', '상무', '불펜', '상황'}

[단독] 태권도, UFC처럼 '격투기 대회' 만든다
http://v.media.daum.net/v/20171227185759152
NaN
{'경기', '관람형', '기술', '국기원', '실험', '문체부', '대회', '태권도'}

[풀카운트] 김태군 떠난 NC 안방, '주전 포수'는 누구?
http://v.media.daum.net/v/20171227165359868
['김태군']
{'NC', '야구', '공백', '주전', '케이비리포트', '1군', '포수', '김태군', '백업'}

'파워 쇼케이스' 변우혁, '또 넘어갑니다"
http://v.media.daum.net/v/20171227162659052
NaN
{'대한민국대표', '파워 쇼케이스 월드', '변우혁', '마이애미'}

한화 연봉 협상, 예비 FA 프리미엄은 없다
http:/

['한현민', '짜장면', '권현빈', '윤종신', '홍석천']
{'어머니', '왼손', '차별', '나이지리아', '엄마', '한현민 혼혈'}

[DA:현장] 불참에 수상 제외까지..빛바랜 대한민국 톱스타상 (종합)
http://v.media.daum.net/v/20171228201223485
['윤계상', '이하늬', '범죄도시', '박서준', '꾼']
{'불참', '수상자', '대한민국', '윤계상', '범죄', '시상식'}

[N이슈]이세창♡정하나 공개한 #신혼집 #악플심경 #새로운삶
http://v.media.daum.net/v/20171228105845221
['이세창', '정하나', '결혼', '신혼집', '이혼']
{'마음', '부부', '이혼', '이세창 정하나', '시선', '결혼'}

[결정적장면]'라스' 한현민 "9살 동생 '태양의 후예'서 송혜교와 연기"
http://v.media.daum.net/v/20171228000115074
['송혜교', '라디오스타', '모모랜드', '권현빈', '홍석천']
{'MBC 라디오스타', '태양의 후예', '동생', '한현민'}

여유도 없고 힐링도 없는데.. <강식당> 왜 인기 있을까
http://v.media.daum.net/v/20171228113259788
['강호동', '이수근', '송민호', '이서진', 'tvn']
{'송민호', '멤버들', '이수근', '윤식당', '강호동', '강식당', 'tvN'}

[단독] 김민석♥이주빈, 동갑내기 열애..일본 후쿠오카 동반여행도
http://v.media.daum.net/v/20171228100002771
['후쿠오카', '데이트', '미스터선샤인', '모델', '이번생은처음이라']
{'촬영', '김민석 이주빈', '후쿠오카', '미모', '모델', '데이트'}

[Oh!쎈 초점] 또 흘러간 국민 걸그룹의 한 시대
http://v.media.daum.net/v/20171228114143123
['원더걸스', '소녀시대', '해체', '씨스타', '

['앙헬 디 마리아']
{'마리아의', '바르셀로나 이적', '레알', '활약'}

[인터뷰] 후련한 양현종 "이제 진짜 야구만 하겠다"
http://v.media.daum.net/v/20171228174403123
['양현종']
{'야구', '28일', 'KIA', '연봉', '계약', '협상', '단장', '크리스마스', '양현종'}

[유럽축구 돋보기] 실바에게 무슨 일이?..18연승 맨시티의 유일한 미스터리
http://v.media.daum.net/v/20171228160136985
['다비드 실바']
{'실바의', '다비드 실바', '프리미어리그', '18연승', '맨시티', '스페인'}

롯데의 2차 드래프트 전력, 실질적 기대치는?
http://v.media.daum.net/v/20171228110203430
NaN
{'투수', '이병규', '자원', '드래프트', '좌완', '고효준', '롯데', '오현택'}

'역시 승부사' 조계현 단장, 양현종의 마음을 잡은 비결
http://v.media.daum.net/v/20171228171913152
['양현종', '조계현']
{'조계현', 'KIA', '타이거즈', '협상', '단장', '재계약', '양현종'}

[오피셜] 리버풀, 반 다이크 영입 발표..등번호 4번
http://v.media.daum.net/v/20171228030139781
['리버풀 FC', '반 다이크']
{'다이크 리버풀', '파운드', '000만', '다이크 영입', '사우샘프턴'}

[풀카운트] 정의윤, 옵션 12억 채우려면 삼진을 줄여야
http://v.media.daum.net/v/20171228162959275
['정의윤']
{'타율', '홈런', '기록', 'SK', '비율', '타점', '계약', '정의윤', '삼진'}

'점검 또 점검' 두산, 마지막까지 치열했던 유재유 지명 순간
http://v.media.daum.net/v/20171228060002942
['유재유']
{'투수', '잠재력', '김현수',

KeyError: '5a46ad6f588c13390875e8b3'

In [ ]:

setto = set()
for idx in list(itertools.combinations(keywords, 2)):
    f1 = ''.join(re.findall('['+idx[0]+']',idx[1])).strip()
    if f1 == 0:pass
    else:
        f2 = min(idx, key = len)
        if abs(len(f1)-len(f2)) == 0:
            setto.add(f2)
setto

In [439]:
list(filter(lambda idx: abs(len(''.join(re.findall('['+idx[0]+']',idx[1])).strip())-len(min(idx, key = len))) == 0, list(itertools.combinations(keywords, 2))))

[]

In [432]:
keywords

{'김가연과 임요환 부부'}

[('식당', '윤식당'), ('윤식당2', '윤식당'), ('제작', '제작진')]

In [372]:
a

('방송', '여행')